In [46]:
import pandas as pd
import warnings
import os
import concurrent.futures

warnings.filterwarnings('ignore')
df = pd.read_csv('data_live_engineer_filtered.csv', encoding='utf-8')


In [47]:
df['result'].value_counts()

result
0    396313
1     33420
Name: count, dtype: int64

In [48]:
df.columns

Index(['minute', 'homeTeam', 'awayTeam', 'goal_home', 'goal_away',
       'shots_home', 'shots_away', 'blockedShots_home', 'blockedShots_away',
       'league', 'corners_home', 'corners_away', 'redcards_home',
       'redcards_away', 'shotsOffgoal_home', 'shotsOffgoal_away',
       'shotsOngoal_home', 'shotsOngoal_away', 'yellowcards_home',
       'yellowcards_away', 'fouls_home', 'fouls_away', 'offsides_home',
       'offsides_away', 'tackles_home', 'tackles_away', 'result', 'match_id',
       'match_result', 'win_rate_home', 'draw_rate_home', 'loss_rate_home',
       'win_rate_away', 'draw_rate_away', 'loss_rate_away', 'f_attack_home',
       'f_attack_away', 'f_defensive_home', 'f_defensive_away', '05ht_home',
       '15ht_home', '25ht_home', '05ht_away', '15ht_away', '25ht_away',
       'possessiontime_away', 'possessiontime_home', 'shotAccuracy_home',
       'shotAccuracy_away', 'attackPressureOverTime_home',
       'attackPressureOverTime_away', 'aggrressionOverTime_home',
      

In [49]:
df.drop(columns=['match_id',
       'match_result', 'win_rate', 'draw_rate', 'loss_rate'], inplace=True)

KeyError: "['win_rate', 'draw_rate', 'loss_rate'] not found in axis"

In [39]:
import unicodedata

# Define a function to normalize text using unicodedata
def normalize_text_unicode(text):
    text = unicodedata.normalize('NFKD', text)
    text = text.encode('ascii', 'ignore').decode('utf-8')
    return text.lower()

# Apply the function to the relevant columns
df['league'] = df['league'].apply(normalize_text_unicode)
df['homeTeam'] = df['homeTeam'].apply(normalize_text_unicode)
df['awayTeam'] = df['awayTeam'].apply(normalize_text_unicode)
df.head()

,minute,homeTeam,awayTeam,goal_home,goal_away,shots_home,shots_away,blockedShots_home,blockedShots_away,league,...,aggresssionOverTime_away,defensiveEfficacy_home,defensiveEfficacy_away,taklesOverTime_home,taklesOverTime_away,possessionControl,passRisk_home,passRisk_away,TotalCards_home,TotalCards_away
0,1.49,carabobo fc,club universitario de deportes,0.0,0.0,0.0,0.0,0.0,0.0,south america - copa libertadores,...,0.0,0.0,0.0,0.0,0.000000,6.0,0.0,0.0,0.0,0.0
1,1.90,carabobo fc,club universitario de deportes,0.0,0.0,0.0,0.0,0.0,0.0,south america - copa libertadores,...,0.0,0.0,0.0,0.0,0.000000,20.0,0.0,0.0,0.0,0.0
2,2.28,carabobo fc,club universitario de deportes,0.0,0.0,0.0,1.0,0.0,1.0,south america - copa libertadores,...,0.0,0.0,100.0,0.0,0.438596,16.0,0.0,0.0,0.0,0.0
3,3.48,carabobo fc,club universitario de deportes,0.0,0.0,0.0,1.0,0.0,1.0,south america - copa libertadores,...,0.0,0.0,100.0,0.0,0.287356,18.0,0.0,0.0,0.0,0.0
4,3.80,carabobo fc,club universitario de deportes,0.0,0.0,0.0,1.0,0.0,1.0,south america - copa libertadores,...,0.0,0.0,100.0,0.0,0.263158,18.0,0.0,0.0,0.0,0.0


In [51]:
df.columns

Index(['minute', 'homeTeam', 'awayTeam', 'goal_home', 'goal_away',
       'shots_home', 'shots_away', 'blockedShots_home', 'blockedShots_away',
       'league', 'corners_home', 'corners_away', 'redcards_home',
       'redcards_away', 'shotsOffgoal_home', 'shotsOffgoal_away',
       'shotsOngoal_home', 'shotsOngoal_away', 'yellowcards_home',
       'yellowcards_away', 'fouls_home', 'fouls_away', 'offsides_home',
       'offsides_away', 'tackles_home', 'tackles_away', 'result', 'match_id',
       'match_result', 'win_rate_home', 'draw_rate_home', 'loss_rate_home',
       'win_rate_away', 'draw_rate_away', 'loss_rate_away', 'f_attack_home',
       'f_attack_away', 'f_defensive_home', 'f_defensive_away', '05ht_home',
       '15ht_home', '25ht_home', '05ht_away', '15ht_away', '25ht_away',
       'possessiontime_away', 'possessiontime_home', 'shotAccuracy_home',
       'shotAccuracy_away', 'attackPressureOverTime_home',
       'attackPressureOverTime_away', 'aggrressionOverTime_home',
      

In [57]:
gols_columns = ['win_rate_home', 'draw_rate_home', 'loss_rate_home',
                                    'win_rate_away', 'draw_rate_away', 'loss_rate_away', 'f_attack_home',
                                    'f_attack_away', 'f_defensive_home', 'f_defensive_away', '05ht_home',
                                    '15ht_home', '25ht_home', '05ht_away', '15ht_away', '25ht_away']
                    
for gols_c in gols_columns:
    if 'home' in gols_c:
        print(df.tail(1)[gols_c].values[0])
    if 'away' in gols_c:
        print(df.tail(1)[gols_c].values[0])


0.2931696950078113
0.454518585241587
0.2523117197506017
0.2523117197506017
0.454518585241587
0.2931696950078113
0.9936583
1.0612212
0.9423105716705322
1.0063821
0.2605762989303756
0.0569243360708943
0.0075071261253633
0.2543109530663506
0.0446476448508452
0.0064629018146925


In [41]:
# Arredondando todos os valores decimais para 2 casas decimais
df = df.round(4)

In [42]:
df.shape

(429733, 66)

In [52]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE

# Encode categorical variables
label_encoders = {}
for column in ['homeTeam', 'awayTeam']:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Split the df into features and target variable
leagues = df['league'].unique()

for league in leagues:
    print(league)
    df_league = df[df['league'] == league]
    X = df_league.drop(['result', 'league', 'match_id'], axis=1)
    y = df_league['result']

    # smote = SMOTE(random_state=42)
    # Split the df into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # X_train, y_train = smote.fit_resample(X_train, y_train)

    # Normalize numerical features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Train a Random Forest classifier
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)

    # Make predictions and evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    print(f"\n{league} - Accuracy: {accuracy}\n")
    print(report)
# X = df.drop(['result'], axis=1)
# y = df['result']

# # from imblearn.over_sampling import SMOTE

# # # Apply SMOTE to the training data
# # smote = SMOTE(random_state=42)

# # Split the df into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# X_train, y_train = smote.fit_resample(X_train, y_train)

# # Normalize numerical features
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# # Train a Random Forest classifier
# model = RandomForestClassifier(random_state=42)
# model.fit(X_train, y_train)

# # Make predictions and evaluate the model
# y_pred = model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# report = classification_report(y_test, y_pred)

South America - Copa Libertadores

South America - Copa Libertadores - Accuracy: 0.9624505928853755

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      2310
           1       0.91      0.63      0.74       220

    accuracy                           0.96      2530
   macro avg       0.94      0.81      0.86      2530
weighted avg       0.96      0.96      0.96      2530

Asia - AFC Champions League

Asia - AFC Champions League - Accuracy: 0.9579271421241662

              precision    recall  f1-score   support

           0       0.96      0.99      0.98      1797
           1       0.90      0.52      0.66       152

    accuracy                           0.96      1949
   macro avg       0.93      0.76      0.82      1949
weighted avg       0.96      0.96      0.95      1949

Brazil - Paulista A1

Brazil - Paulista A1 - Accuracy: 0.9646924829157175

              precision    recall  f1-score   support

           0       0.97   

KeyboardInterrupt: 

In [54]:
for league in leagues:
    print(league)
    df_league = df[df['league'] == league]
    X = df_league.drop(['result', 'league', 'match_id'], axis=1)
    y = df_league['result']

    smote = SMOTE(random_state=42)
    # Split the df into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    X_train, y_train = smote.fit_resample(X_train, y_train)

    # Normalize numerical features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Train a Random Forest classifier
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)

    # Make predictions and evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    print(f"\n{league} - Accuracy: {accuracy}\n")
    print(report)

South America - Copa Libertadores

South America - Copa Libertadores - Accuracy: 0.967193675889328

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      2310
           1       0.87      0.73      0.80       220

    accuracy                           0.97      2530
   macro avg       0.92      0.86      0.89      2530
weighted avg       0.97      0.97      0.97      2530

Asia - AFC Champions League

Asia - AFC Champions League - Accuracy: 0.9671626475115443

              precision    recall  f1-score   support

           0       0.97      0.99      0.98      1797
           1       0.87      0.68      0.76       152

    accuracy                           0.97      1949
   macro avg       0.92      0.83      0.87      1949
weighted avg       0.97      0.97      0.97      1949

Brazil - Paulista A1

Brazil - Paulista A1 - Accuracy: 0.9698177676537585

              precision    recall  f1-score   support

           0       0.98    

KeyboardInterrupt: 

In [30]:
X.columns

Index(['minute', 'homeTeam', 'awayTeam', 'goal_home', 'goal_away',
       'shots_home', 'shots_away', 'blockedShots_home', 'blockedShots_away',
       'corners_home', 'corners_away', 'redcards_home', 'redcards_away',
       'shotsOffgoal_home', 'shotsOffgoal_away', 'shotsOngoal_home',
       'shotsOngoal_away', 'yellowcards_home', 'yellowcards_away',
       'fouls_home', 'fouls_away', 'offsides_home', 'offsides_away',
       'tackles_home', 'tackles_away', 'homeTeam_win', 'homeTeam_loss',
       'homeTeam_draw', 'win_rate_homeTeam', 'loss_rate_homeTeam',
       'draw_rate_homeTeam', 'awayTeam_win', 'awayTeam_loss', 'awayTeam_draw',
       'win_rate_awayTeam', 'loss_rate_awayTeam', 'draw_rate_awayTeam',
       'f_attack_home', 'f_attack_away', 'f_defensive_home',
       'f_defensive_away', '05ht_home', '15ht_home', '25ht_home', '05ht_away',
       '15ht_away', '25ht_away', 'possessiontime_away', 'possessiontime_home',
       'shotAccuracy_home', 'shotAccuracy_away', 'attackPressureOverT

In [31]:
print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       994
           1       0.97      0.95      0.96        40

    accuracy                           1.00      1034
   macro avg       0.99      0.97      0.98      1034
weighted avg       1.00      1.00      1.00      1034



In [32]:
print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       994
           1       0.97      0.95      0.96        40

    accuracy                           1.00      1034
   macro avg       0.99      0.97      0.98      1034
weighted avg       1.00      1.00      1.00      1034



In [33]:
import joblib

# Save the label encoders, scaler, and model
joblib.dump(label_encoders, '../models/label_encoders.pkl')
joblib.dump(scaler, '../models/scaler.pkl')
joblib.dump(model, '../models/random_forest_model.pkl')

['../models/random_forest_model.pkl']